## Big Data

Big Data se refiere a las técnicas y herramientas utilizadas para procesar grandes volúmenes de datos que no pueden ser manejados por una sola computadora debido a su volumen, velocidad, variedad y veracidad (las 4 "V").

- **Volumen:** Cantidad masiva de datos.

- **Velocidad:** Rapidez con la que se generan y procesan los datos.

- **Variedad:** Diferentes tipos de datos (estructurados, semi-estructurados, no estructurados).

- **Veracidad:** Calidad y confiabilidad de los datos.

## Ecosistema de Apache Hadoop

Apache Hadoop es un framework de código abierto diseñado para el procesamiento distribuido de grandes volúmenes de datos en clústeres de computadoras. Hadoop se compone de varios módulos, entre los que destacan:

- **HDFS (Hadoop Distributed File System):** Sistema de archivos distribuido que almacena datos en múltiples nodos.

- **MapReduce:** Modelo de programación para procesar grandes conjuntos de datos en paralelo.

- **YARN (Yet Another Resource Negotiator):** Gestor de recursos que administra los recursos del clúster y planifica las tareas.

![Ecosistena_Apache_Hadoop](Ecosistema_Apache_Hadoop.png)

#### 4. **Apache Hive**

Apache Hive es una herramienta construida sobre Hadoop que permite realizar consultas SQL sobre grandes volúmenes de datos almacenados en HDFS. Hive convierte las consultas SQL en trabajos de MapReduce, lo que facilita el análisis de datos para usuarios familiarizados con SQL.

**Integración de Hive con Spark**

Spark puede leer y escribir datos en tablas de Hive, lo que permite combinar la potencia de Spark con la facilidad de uso de Hive para consultas SQL.

### Data Lakehouse
Un Data Lakehouse es un repositorio centralizado que almacena grandes cantidades de datos en su formato crudo, tanto estructurados como no estructurados. Combina las ventajas de un Data Lake (almacenamiento de datos en bruto) con las de un Data Warehouse (gestión de datos estructurados).

![Data Lakehouse](datalakehouse.png)

##  Resilient Distributed Datasets (RDDs) en Spark

### 1. Introducción a los RDD en Spark
Los Resilient Distributed Datasets (RDDs) son la estructura de datos fundamental en Apache Spark. Representan una colección inmutable y distribuida de elementos que pueden procesarse en paralelo. Características clave:

- **Resilientes**: Tolerantes a fallos mediante recomputación.

- **Distribuidos**: Los datos se dividen en particiones y se distribuyen en el clúster.

- **Inmutables**: No se modifican, pero se transforman en nuevos RDDs.

- **Lazy Evaluation (Evaluación Perezosa)**:
  
   - Las transformaciones en RDD no se ejecutan inmediatamente
  
   - Spark espera hasta que se requiere una acción (como collect o count) para ejecutar las operaciones, optimizando así el procesamiento.

### 2. Tipos de RDDs
- **RDDs simples**: Colecciones de elementos sin esquema.

- **RDDs clave-valor (Pair RDDs)**: Colecciones de pares (clave, valor).

- **RDDs numéricos**: RDDs que contienen datos numéricos para operaciones estadísticas.

**Ejemplos**:

In [1]:
//Crea un RDD con una lista de números enteros.
val sc = spark.sparkContext
val rddSimple = sc.parallelize(Seq(1, 2, 3, 4, 5))
rddSimple.collect().foreach(println)

1
2
3
4
5


sc = org.apache.spark.SparkContext@55fc865d
rddSimple = ParallelCollectionRDD[0] at parallelize at <console>:27


ParallelCollectionRDD[0] at parallelize at <console>:27

In [2]:
//Crea un RDD donde cada elemento es un par (clave, valor)
val rddPair = sc.parallelize(Seq(("A", 1), ("B", 2), ("C", 3)))
rddPair.collect().foreach(println)

(A,1)
(B,2)
(C,3)


rddPair = ParallelCollectionRDD[1] at parallelize at <console>:25


ParallelCollectionRDD[1] at parallelize at <console>:25

In [4]:
//Se usa la extensión RDDFunctions para cálculos numéricos

import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.rdd.RDDFunctions._

val rddNumerico = sc.parallelize(Seq(10.5, 20.3, 30.7, 40.1))
val media = rddNumerico.mean()
println(s"Media: $media")

Media: 25.4


rddNumerico = ParallelCollectionRDD[4] at parallelize at <console>:35
media = 25.4


25.4

### 3. ¿Cuándo es Conveniente Usar RDDs?
Los RDDs son útiles cuando:

- Se necesita un control fino sobre las operaciones de bajo nivel.

- Se trabaja con datos no estructurados o semi-estructurados.

- Se requieren transformaciones complejas que no están soportadas en DataFrames o DataSets.

### 4. Diferencias entre DataFrames vs DataSets vs RDDs

| Característica   | RDDs                         | DataFrames                   | DataSets                          |
|-----------------|-----------------------------|------------------------------|-----------------------------------|
| Esquema        | Sin esquema                  | Con esquema                  | Con esquema (tipado fuerte)      |
| Optimización   | Manual                       | Automática (Catalyst)        | Automática (Catalyst)            |
| API            | Funcional (Scala, Java, Python) | SQL-like                      | Tipado fuerte (Scala, Java)      |
| Rendimiento    | Menor (sin optimización)     | Mayor (optimizado)           | Mayor (optimizado y tipado)      |


### 5. Creación de un RDD
Los RDDs se pueden crear de varias formas:

- **Desde una colección local**:

In [5]:
val rdd = sc.parallelize(Seq(1, 2, 3, 4, 5))

rdd = ParallelCollectionRDD[6] at parallelize at <console>:32


ParallelCollectionRDD[6] at parallelize at <console>:32

- **Desde archivos externos**:

In [6]:
val rdd = sc.textFile("ruta/al/archivo.txt")

rdd = ruta/al/archivo.txt MapPartitionsRDD[8] at textFile at <console>:32


ruta/al/archivo.txt MapPartitionsRDD[8] at textFile at <console>:32

### 6. Interoperabilidad entre RDDs, DataSets y DataFrames
- **De RDD a DataFrame**:

In [11]:
//toDF() funciona solo si import spark.implicits._ está presente.
import org.apache.spark.sql.SparkSession
import spark.implicits._

val rdd = spark.sparkContext.parallelize(Seq("A", "B", "C"))
val df = rdd.toDF("value") // Se convierte en DataFrame con una columna llamada "value"

df.show()

+-----+
|value|
+-----+
|    A|
|    B|
|    C|
+-----+



rdd = ParallelCollectionRDD[18] at parallelize at <console>:43
df = [value: string]


[value: string]

- **De DataFrame a RDD**:

In [12]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder.appName("Ejemplo").getOrCreate()
import spark.implicits._

// Crear un DataFrame
val df = Seq(("Ana", 30), ("Juan", 25)).toDF("nombre", "edad")

// Convertir DataFrame a RDD
val rdd = df.rdd

// Imprimir el contenido del RDD
rdd.collect().foreach(println)


[Ana,30]
[Juan,25]


spark = org.apache.spark.sql.SparkSession@700a548a
df = [nombre: string, edad: int]
rdd = MapPartitionsRDD[26] at rdd at <console>:52


MapPartitionsRDD[26] at rdd at <console>:52

- **De RDD a DataSet**:

In [10]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.Row

val spark = SparkSession.builder.appName("Ejemplo").getOrCreate()
import spark.implicits._

// Crear un RDD de una case class
case class Persona(nombre: String, edad: Int)
val rdd = spark.sparkContext.parallelize(Seq(Persona("Ana", 30), Persona("Juan", 25)))

// Convertir RDD a Dataset
val ds = rdd.toDS()
ds.show()

+------+----+
|nombre|edad|
+------+----+
|   Ana|  30|
|  Juan|  25|
+------+----+



spark = org.apache.spark.sql.SparkSession@700a548a
defined class Persona
rdd = ParallelCollectionRDD[14] at parallelize at <console>:41
ds = [nombre: string, edad: int]


[nombre: string, edad: int]

In [13]:
val rddTipado = df.rdd.map(row => (row.getString(0), row.getInt(1)))
rddTipado.collect().foreach(println)

(Ana,30)
(Juan,25)


rddTipado = MapPartitionsRDD[27] at map at <console>:47


MapPartitionsRDD[27] at map at <console>:47

### 7. Creación de un RDD a partir de una Colección Local

In [16]:
val datos = Seq(1, 2, 3, 4, 5)
val rdd = sc.parallelize(datos)

// Imprimir el contenido del RDD
rdd.collect().foreach(println) //collect(): Recoge todos los elementos del RDD y los devuelve como un array en el driver.
                               //foreach(println): Itera sobre los elementos del array y los imprime.

1
2
3
4
5


datos = List(1, 2, 3, 4, 5)
rdd = ParallelCollectionRDD[30] at parallelize at <console>:50


ParallelCollectionRDD[30] at parallelize at <console>:50

### 8. Creación de un RDD de Distintos Orígenes
- **Desde un archivo de texto**:

In [ ]:
val rdd = sc.textFile("ruta/al/archivo.txt")

- **Desde HDFS**:

In [ ]:
val rdd = sc.textFile("hdfs://ruta/al/archivo.txt")

- **Desde una base de datos**:

In [ ]:
val rdd = sc.jdbcRDD(...)

### 9. Manipulación de RDDs
**a) Transformaciones**
Las transformaciones crean un nuevo RDD a partir de uno existente:

- **map**: Aplica una función a cada elemento.

In [ ]:
val rdd2 = rdd.map(x => x * 2)

- **filter**: Filtra elementos según una condición.

In [ ]:
val rdd2 = rdd.filter(x => x > 2)

- **flatMap**: Aplica una función que devuelve múltiples elementos.

In [ ]:
val rdd2 = rdd.flatMap(x => Seq(x, x * 2))

**b) Acciones**
Las acciones devuelven un valor o escriben datos:

- **count**: Cuenta el número de elementos.

In [ ]:
val total = rdd.count()

- **collect**: Recupera todos los elementos al driver.

In [ ]:
val datos = rdd.collect()

- **reduce**: Combina elementos usando una función.

In [ ]:
val suma = rdd.reduce((x, y) => x + y)

### 10. Guardado de Archivos
- **Archivos de texto**:

In [ ]:
rdd.saveAsTextFile("ruta/de/salida")

- **Archivos Hadoop**:

In [ ]:
rdd.saveAsHadoopFile(...)

- **Otros formato**:

In [ ]:
rdd.saveAsObjectFile("ruta/de/salida")

### 11. Cacheo de RDDs
El cacheo permite almacenar RDDs en memoria o disco para reutilizarlos:

In [ ]:
rdd.cache()  // Almacena en memoria
rdd.persist(StorageLevel.DISK_ONLY)  // Almacena en disco

### 12. Establecimiento de Puntos de Control
Los puntos de control (checkpoints) permiten guardar el estado de un RDD en un almacenamiento confiable:

In [ ]:
sc.setCheckpointDir("ruta/de/checkpoint")
rdd.checkpoint()

#### **Conclusión**

Los RDDs son la base de Apache Spark y ofrecen un control fino sobre el procesamiento distribuido. Aunque DataFrames y DataSets son más modernos y optimizados, los RDDs siguen siendo útiles para casos específicos que requieren operaciones de bajo nivel. Con un manejo adecuado de transformaciones, acciones y técnicas como el cacheo, se pueden construir aplicaciones eficientes y escalables.

## **Apache Spark**

#### **1. ¿Qué es Apache Spark?**

Apache Spark es un motor de procesamiento de datos distribuido y en memoria, diseñado para ser rápido y eficiente en el manejo de grandes volúmenes de datos. A diferencia de Hadoop MapReduce, que escribe y lee datos del disco en cada paso, Spark mantiene los datos en memoria, lo que lo hace hasta 100 veces más rápido para ciertas cargas de trabajo.

#### **Ventajas de Spark sobre Hadoop MapReduce:**

- **Velocidad:** Al procesar datos en memoria, Spark evita la sobrecarga de E/S (entrada/salida) en disco, lo que lo hace ideal para aplicaciones que requieren baja latencia.

- **Facilidad de uso:** Spark ofrece APIs en Scala, Java, Python y R, lo que lo hace accesible para una amplia gama de desarrolladores.

- **Unificación:** Spark integra múltiples componentes (SQL, streaming, machine learning, procesamiento de grafos) en una sola plataforma, eliminando la necesidad de usar herramientas separadas.

- **Tolerancia a fallos:** Spark utiliza RDD (Resilient Distributed Datasets), que son colecciones de datos distribuidos y tolerantes a fallos.

#### **Casos de uso en ciencia de datos:**

- **Procesamiento de datos a gran escala:** Limpieza, transformación y análisis de grandes volúmenes de datos.

- **Machine Learning:** Entrenamiento de modelos con grandes conjuntos de datos usando MLlib.

- **Streaming en tiempo real:** Procesamiento de flujos de datos en tiempo real con Spark Streaming.

- **Análisis de grafos:** Procesamiento de datos en forma de grafos con GraphX.

#### **2. Componentes principales de Spark**

Spark es una plataforma modular que incluye varios componentes para diferentes tipos de procesamiento de datos:

**1. Spark Core:**

- Es el núcleo de Spark y proporciona la funcionalidad básica para la ejecución distribuida de tareas.

- Incluye soporte para RDD, que son la base de todas las operaciones en Spark.

**2. Spark SQL:**

- Permite trabajar con datos estructurados usando consultas SQL.

- Proporciona una interfaz unificada para trabajar con DataFrames y Datasets.

- Ejemplo: Consultas SQL sobre grandes volúmenes de datos almacenados en HDFS o bases de datos.

**3. Spark Streaming:**

- Permite procesar flujos de datos en tiempo real.

- Ejemplo: Análisis de logs en tiempo real o procesamiento de datos de sensores.

**4. MLlib (Machine Learning Library):**

- Biblioteca de machine learning escalable que incluye algoritmos como clasificación, regresión, clustering y recomendación.

- Ejemplo: Entrenamiento de modelos predictivos con grandes conjuntos de datos.

**5. GraphX:**

- Biblioteca para el procesamiento de grafos y análisis de redes.

- Ejemplo: Análisis de redes sociales o detección de comunidades en grafos.